In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
import multiprocessing as mp
import rasterio

from pathlib import Path
from rasterio.plot import show

%matplotlib inline

In [ ]:
data_path = Path('.').resolve().parent / "data"

In [ ]:
urban_network_old = gpd.read_file(data_path / 'Oteiza' / 'urban_network' / 'Calles Plan.shp')

In [ ]:
elevation = rasterio.open(data_path  / 'urban_network'/ 'elevation.asc')

In [ ]:
shelter_locations = gpd.read_file(data_path  / 'Oteiza' / 'shelter_locations' / 'P_E.shp')
shelter_locations.head()

In [ ]:
population_distribution_old = gpd.read_file(data_path / "Oteiza" / "population_distribution" / "u_3_Diurno.shp")
population_distribution_old.head()

In [ ]:
tsunami_inundatation = rasterio.open(data_path  / 'tsunami_inundation'/ 'sample.asc')

## Urban Network

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))
urban_network_old.to_crs('epsg:4326').plot(ax=ax)
fig.show()

In [ ]:
urban_area = urban_network_old.to_crs('epsg:4326').convex_hull.iloc[0]
G = ox.graph_from_polygon(
    urban_area,
    network_type='walk',
    simplify=False
)
G = ox.project_graph(G, to_crs='epsg:32719')
ox.add_node_elevations_raster(
    G,
    data_path  / 'urban_network'/ 'elevation.asc',
    band=1
)
ox.elevation.add_edge_grades(G, add_absolute=False, precision=3)
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)

### Shortest path

In [ ]:
def euclidean_dist(a, b, G=G):
    x1 = G.nodes[a]['x']
    y1 = G.nodes[a]['y']
    x2 = G.nodes[b]['x']
    y2 =  G.nodes[b]['y']
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5


def best_astar_path(G, source, targets, heuristic=None, weight='weight'):
    paths = {target: nx.astar_path(G, source, target, heuristic, weight) for target in targets}
    lengths = {target: sum(ox.utils_graph.get_route_edge_attributes(G, path, weight)) for target, path in paths.items()}
    best_target = min(lengths, key=lengths.get)
    return (source, best_target, lengths[best_target], paths[best_target])

In [ ]:
shelter_nodes = ox.distance.nearest_nodes(
    G,
    X=shelter_locations['x'].values,
    Y=shelter_locations['y'].values
)

In [ ]:
cpus = None
args = ((G, origin, shelter_nodes, euclidean_dist, 'length') for origin in G.nodes)
pool = mp.Pool(cpus)
sma = pool.starmap_async(best_astar_path, args)
paths = sma.get()
pool.close()
pool.join()

In [ ]:
path_df = pd.DataFrame(paths, columns=['osmid', 'shelter_osmid', 'cost', 'route_list'])
# path_df['route'] = path_df['route_list'].apply(lambda route: '&'.join(str(x) for x in route))
path_df['next_node'] = path_df['route_list'].apply(lambda x: x[1] if len(x) > 1 else -9999)
path_df.head()

In [ ]:
# test
# path_df.loc[lambda x: x["osmid"].isin(path_df.loc[0, "route_list"])]

In [ ]:
netlogo_nodes = (
    nodes.reset_index()
    .merge(
        path_df.drop(columns='route_list'),
        on='osmid'
    )
    .rename(columns={"osmid": "id", "shelter_osmid": "shelter_id"})
    .loc[: ,["id", "x", "y", "shelter_id", "next_node", "geometry"]]
)
netlogo_nodes.head()

In [ ]:
netlogo_urban_network = (
    edges.reset_index()
    .rename(
        columns={
            "osmid": "id",
            "u": "from_id",
            "v": "to_id",
            "grade": "slope",
        }
    )
    .assign(weight=lambda x: x["length"])
    .loc[
        :,
        [
            "id",
            "from_id",
            "to_id",
            "slope",
            "weight",
            "length",
            "highway",
            "lanes",
            "geometry"

        ]
    ]
)
netlogo_urban_network.head()

In [ ]:
netlogo_nodes.to_file(data_path / 'urban_network'/ "nodes.shp")
netlogo_urban_network.to_file(data_path / 'urban_network'/ "urban_network.shp")

## Shelters

In [ ]:
netlogo_shelters = (
    netlogo_nodes.loc[lambda x: x["id"].isin(shelter_nodes)]
    .assign(evac_type="horizontal")
    .loc[
        :,
        ["id", "x", "y", "evac_type", "geometry"]
    ]
)
netlogo_shelters.head()

In [ ]:
netlogo_shelters.to_file(data_path / 'shelters'/ "shelters.shp")

## Agent Distribution

In [ ]:
netlogo_population_distribution = (
    population_distribution_old.rename(
        columns={
            "Agente": "id",
            "Velocidad": "speed",
            "td": "depar_time",
        }
    )
    .assign(age=99)
    .loc[
        :,
        ["id", "age", "depar_time", "speed", "geometry"]
    ]
)
netlogo_population_distribution.head()

In [ ]:
netlogo_population_distribution.to_file(data_path / 'agent_distribution'/ "agent_distribution.shp")

## Tsunami

In [ ]:
show((tsunami_inundatation, 1))